In [1]:
import math

import pandas as pd
from matplotlib import pyplot as plt
from opinionmining import *


KeyboardInterrupt



In [ ]:
paths = []
for folder in os.listdir(full_path):
    try:
        for file in os.listdir(os.path.join(full_path, folder)):
            if str(file) != "Readme.txt" and str(file) != ".DS_Store":
                paths.append(os.path.join(full_path, folder, file))
    except NotADirectoryError:
        continue
paths[:3]

In [ ]:
SAVE = False
pwd = os.getcwd()
filename = "db.csv"
if SAVE:
    db = ReviewDatabase(paths).dataframe.to_csv(os.path.join(pwd, filename))
else:
    db = pd.read_csv(filename)

database = db.where(db["Product_ID"] == 1).copy().dropna()


In [ ]:
database.Sentence

In [ ]:

database["ExtractedCategories"] = pd.Series(map(lambda x: FeatureExtraction.categories(x), database.Sentence))


What is the product?
What are the opinion categories?
What is the sentiment of the opinion categories?

In [ ]:
review_1 = database.where(database["Product_ID"] == 1)
review_1

In [ ]:
#Create a copy of the database to perform feature extraction
category_table = database.loc[:,"Product_ID":"Sentence_ID"].copy()
# Perform Stemming on the sentences
category_table["Stemmed_Sentence"] = database.Sentence.apply(lambda x: FeatureExtraction.stemming([x])[0])
# Remove the stop words
category_table["Stemmed_Sentence"] = category_table["Stemmed_Sentence"].apply(lambda x: FeatureExtraction.remove_stop([x])[0])

category_table["FuzzyMatchedCategories"] = database["ExtractedCategories"].apply(lambda x: FeatureExtraction.fuzzy_match_categories())
# category_table["Stemmed"] = category_table["ExtractedCategories"].apply(lambda x : FeatureExtraction.stemming(x))
# display(category_table)
# # Flatten the stemmed categories into a list and then count the frequencies of each category
# flattened_list =  [category for category_set in category_table["Stemmed"] for category in category_set]
# counter = Counter(flattened_list)
# counts = [c for p, c in counter.most_common()]
# items = [p for p, c in counter.most_common()]
# # counts
# counter.most_common(10)


In [ ]:

        
# category_table["fuzzymatched_topics"] = pd.Series(map(lambda x: FeatureExtraction.fuzzy_match_categories(x, searchable_topics), category_table.Stemmed))


topic_vocabulary = set([word for sublist in category_table.fuzzymatched_topics for word in sublist])
# Initialize the vectorizer
vectorizer = CountVectorizer(vocabulary=topic_vocabulary, binary=True)

# Apply the vectorizer
vector_matrix = vectorizer.fit_transform(category_table['fuzzymatched_topics'].apply(lambda x: " ".join(x)))
# vector_df = pd.DataFrame(vector_matrix.toarray(), columns=vectorizer.get_feature_names_out())
# category_table['topic_vectors'] = vector_df.values.tolist()


In [ ]:
category_table

The difficulty here is in mapping between effectively a many to one (through the process of stemming) to a one to one, where each category can be read as the most common form of that category.  Relies on explicitly stated product categories. 

Use unsupervised learning to match stemmed review topics to broader content topics
- do this by vectorizing the 

In [ ]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vocabulary = category_table.Stemmed_Sentence.to_numpy().flatten()
# 
# # Applying KMeans
# kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
# 
# # Predicting the clusters
# labels = kmeans.predict(X)
# 
# # Plotting
# plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis')
# plt.show()

In [ ]:
vocabulary